In [5]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import math
from scipy.optimize import curve_fit
import glob
import os
import matplotlib.backends.backend_pdf

In [6]:
file_name = []

for file in os.listdir("./"):
    if file.endswith(".fits"):
        file_name.append(file)

In [7]:
f = open('line_list3.dat', 'r')
g = open('Instrument_list.dat','r')
h = open('speclist_uves.dat', 'r')


#delta_v = 1000.0;
c = 300000.0;
line_name = []
line_lam = []
delta_lam = []
lambda_over_c = []
line_num = 0

# Getting wavelengths and names of the important lines
for line in f.readlines():
    row = line.strip()
    columns = row.split()
    line_lam.append(float(columns[0]))
    line_name.append(columns[1]+columns[2])
    #lambda_over_c.append(float(columns[0])/c) # delta_lambda = lambda*(delta_v)/c
    #print (delta_v*float(columns[0])/c)
    line_num = line_num + 1

f.close()
line_wavelength = np.array(line_lam)

In [8]:
instrument = []
resolution = []
plot_window = []

for line in open('Instrument_list.dat'):
    row = line.strip()
    if not row.startswith("#"):
        columns = row.split()
        instrument.append(columns[0] + ' ' + columns[1])
        resolution.append(float(columns[2]))
        plot_window.append(float(columns[3]))
        
        
grb_num = 0;
grb_file_name = []
grb_redshift = []
grb_instrument = []
#for line in open('speclist.dat'):
#for i in range (0, 65):
#    h.readline()
    
pdf = matplotlib.backends.backend_pdf.PdfPages("GRB_Plots_050730.pdf")

    
for line in h.readlines():
    row=line.strip()
    if not row.startswith("#"):
        columns = line.split()
        grb_file_name.append(columns[0])
        grb_redshift.append(float(columns[1]))
        grb_instrument.append(columns[2] + ' ' + columns[3])
        
        print(columns[0])
        
        hdulist = fits.open(grb_file_name[grb_num])
        Flux = hdulist[0]
        Error = hdulist[1]
        Wavelength = hdulist[2]
        
        redshift = grb_redshift[grb_num] #Getting the GRB Redshift
        Wavelength_rest = np.array(Wavelength.data/(1+redshift)) # Transforming to rest wavelength
        Flux_norm = np.array(Flux.data)
        Error_norm = np.array(Error.data)
        
        instru_num = instrument.index(grb_instrument[grb_num]) #Finding the index of the array corresponding to the particular instrument used
        instru_res = resolution[instru_num] # Getting Instrument resolution, if it is needed in future
        delta_v = plot_window[instru_num]  # Getting the corresponding plot window as written in Instrument_list.dat 
        
        delta_lam = delta_v*line_wavelength/c # delta_lambda = lambda*(delta_v)/c
        delta_lambda = np.array(delta_lam)
        
        #Plotting the lines which are present in the data subplot of 'nrows' rows and 'ncols' columns
        Wavelength_box = []
        Velocity_box = []
        Flux_box = []
        Error_box = []

        fig_num = 4
        
        for fig_count in range (0, fig_num):
            ncols = 5
            nrows = int(math.ceil(float(line_num)/float(ncols*fig_num)))

            f, axarr = plt.subplots(nrows, ncols)
        
            plot_name = grb_file_name[grb_num]
            plot_name = plot_name[:-5]
            plt.suptitle((plot_name +' z = '+str(redshift) + ' ' + str(grb_instrument[grb_num]) ) , fontsize = 30)
            plot_name = plot_name + '.png'
        
            for i in range(fig_count*nrows*ncols, min( line_num, (fig_count+1)*nrows*ncols) ):

                Wavelength_window = Wavelength_rest[(Wavelength_rest >= (line_wavelength[i]-delta_lambda[i])) * (Wavelength_rest <= (line_wavelength[i]+delta_lambda[i]))]
                Velocity_window = ((Wavelength_window - line_wavelength[i])*c)/line_wavelength[i]
                Flux_window = Flux_norm[(Wavelength_rest >= (line_wavelength[i]-delta_lambda[i])) * (Wavelength_rest <= (line_wavelength[i]+delta_lambda[i]))]
                Error_window = Error_norm[(Wavelength_rest >= (line_wavelength[i]-delta_lambda[i])) * (Wavelength_rest <= (line_wavelength[i]+delta_lambda[i]))]
            
                Wavelength_box.append(Wavelength_window)
                Velocity_box.append(Velocity_window)
                Flux_box.append(Flux_window)
                Error_box.append(Error_window)
        
                rownum = int((i-fig_count*nrows*ncols)%nrows)
                colnum = int((i-fig_count*nrows*ncols)//nrows)

                axarr[rownum, colnum].errorbar(Velocity_window, Flux_window, yerr=Error_window, ecolor='r')
                axarr[rownum, colnum].set_ylim(-1.5, 1.5) #axarr[rownum, colnum].set_ylim(-0.1, 1.5)
                axarr[rownum, colnum].text(0, 1.3, line_name[i], fontsize=20)
                #axarr[rownum, colnum].set_xlabel('Velocity (km/s)', fontsize=15)
                #axarr[rownum, colnum].set_ylabel('Normalized Flux (arbitrary units)', fontsize=15)
                max_yticks = 5
                max_xticks = 5            
                yloc = plt.MaxNLocator(max_yticks)
                xloc = plt.MaxNLocator(max_xticks)
                axarr[rownum, colnum].yaxis.set_major_locator(yloc)
                axarr[rownum, colnum].xaxis.set_major_locator(xloc)
                axarr[rownum, colnum].tick_params(axis='x', labelsize=15)
                axarr[rownum, colnum].tick_params(axis='y', labelsize=15)
    
                axarr[rownum, colnum].axhline(y=1, color='k', ls='dashed')
                axarr[rownum, colnum].axvline(x=0, color='k', ls='dashed')
        
            
            figure = plt.gcf() # get current figure
            figure.set_size_inches(20, 15)
            # when saving, specify the DPI
            #plt.savefig(plot_name, dpi = 150, bbox_inches='tight')
            pdf.savefig(figure, dpi = 100, bbox_inches='tight')
            plt.close()
        
        # Now the full spectrum and then we'll close
        full_subplot_num = 3
        num_points = len(Wavelength_rest)
        f, axarr = plt.subplots(full_subplot_num)
        plt.suptitle((plot_name +' z = '+str(redshift) + ' ' + str(grb_instrument[grb_num]) ) , fontsize = 30)

        for p in range(0,full_subplot_num):
            x = Wavelength_rest[(num_points/full_subplot_num)*p:(num_points/full_subplot_num)*(p+1)+1]
            y = Flux_norm[(num_points/full_subplot_num)*p:(num_points/full_subplot_num)*(p+1)+1]
            er = Error_norm[(num_points/full_subplot_num)*p:(num_points/full_subplot_num)*(p+1)+1]
            axarr[p].errorbar(x,y,er, ecolor = 'r')        
            axarr[p].set_ylim(-3, 2)
            axarr[p].text(0, 1.3, line_name[i], fontsize=20)
            max_yticks = 8
            max_xticks = 10            
            yloc = plt.MaxNLocator(max_yticks)
            xloc = plt.MaxNLocator(max_xticks)
            axarr[p].yaxis.set_major_locator(yloc)
            axarr[p].xaxis.set_major_locator(xloc)
            axarr[p].tick_params(axis='x', labelsize=15)
            axarr[p].tick_params(axis='y', labelsize=15)    
            axarr[p].axhline(y=1, color='k', ls='dashed')
            
        figure = plt.gcf() # get current figure
        figure.set_size_inches(15, 15)
        # when saving, specify the DPI
        #plt.savefig(plot_name, dpi = 150, bbox_inches='tight')
        pdf.savefig(figure, dpi = 100)
        plt.close()       
        # This is the final step: To increment!! Keep this at the end all the time, otherwise you'll get errors!!
        grb_num = grb_num+1
pdf.close()

050730_uves.fits


In [ ]:
nrows = int(math.ceil(float(line_num)/float(ncols*fig_num)))
print(nrows)

In [ ]:
plt.figure()
plt.errorbar(Wavelength.data, Flux.data, yerr= Error.data, ecolor='r')
plt.title("Simplest Errorbars")
axes = plt.gca()
axes.set_ylim([-3,3]) #axes.set_ylim([-2,2])
#axes.set_xlim([8000,8500])
plt.show()

redshift =  3.35 #2.433 #3.53 #3.258 #redshift = 2.332 #redshift = 4.953
Wavelength_rest = np.array(Wavelength.data/(1+redshift))
Flux_norm = np.array(Flux.data)
Error_norm = np.array(Error.data)

#Sample plot for check
plt.figure()
plt.errorbar(Wavelength_rest, Flux_norm, Error_norm, ecolor = 'r')
#plt.title("Simplest Errorbars")
axes = plt.gca()
axes.set_ylim([-2,2])
axes.set_xlim([1030,1047])
plt.show()

In [ ]:
#For GMOS we will take +/-1000km/s around the center of the line to make Fox et al Plot
f = open('line_list2.dat', 'r')

delta_v = 400; #1000.0;
c = 300000.0;
line_name = []
line_lam = []
delta_lam = []
line_num = 0

# Getting wavelengths and names of the important files
for line in f.readlines():
    row = line.strip()
    columns = row.split()
    line_lam.append(float(columns[0]))
    line_name.append(columns[1]+columns[2])
    delta_lam.append(delta_v*float(columns[0])/c) # delta_lambda = lambda*(delta_v)/c
    #print (delta_v*float(columns[0])/c)
    line_num = line_num + 1

f.close()
line_wavelength = np.array(line_lam)
delta_lambda = np.array(delta_lam)
#print line_num

In [ ]:
#Plotting the lines which are present in the data subplot of 'nrows' rows and 'ncols' columns
Wavelength_box = []
Velocity_box = []
Flux_box = []
Error_box = []

fig_num = 4
for fig_count in range (0, fig_num):
    ncols = 5
    nrows = int(math.ceil(line_num/(ncols*fig_num)))
    f, axarr = plt.subplots(nrows, ncols)
    plt.suptitle(Filename, fontsize = 30)
    
    for i in range(fig_count*nrows*ncols, min( line_num, (fig_count+1)*nrows*ncols) ):
        Wavelength_window = Wavelength_rest[(Wavelength_rest >= (line_wavelength[i]-delta_lambda[i])) * (Wavelength_rest <= (line_wavelength[i]+delta_lambda[i]))]
        Velocity_window = ((Wavelength_window - line_wavelength[i])*c)/line_wavelength[i]
        Flux_window = Flux_norm[(Wavelength_rest >= (line_wavelength[i]-delta_lambda[i])) * (Wavelength_rest <= (line_wavelength[i]+delta_lambda[i]))]
        Error_window = Error_norm[(Wavelength_rest >= (line_wavelength[i]-delta_lambda[i])) * (Wavelength_rest <= (line_wavelength[i]+delta_lambda[i]))]
                
        Wavelength_box.append(Wavelength_window)
        Velocity_box.append(Velocity_window)
        Flux_box.append(Flux_window)
        Error_box.append(Error_window)
        
        rownum = int((i-fig_count*nrows*ncols)%nrows)
        colnum = int((i-fig_count*nrows*ncols)//nrows)
        axarr[rownum, colnum].errorbar(Velocity_window, Flux_window, yerr=Error_window, ecolor='r')
        axarr[rownum, colnum].set_ylim(-1.5, 2) #axarr[rownum, colnum].set_ylim(-0.1, 1.5)
        axarr[rownum, colnum].text(0, 1.3, line_name[i], fontsize=20)
        #axarr[rownum, colnum].set_xlabel('Velocity (km/s)', fontsize=15)
        #axarr[rownum, colnum].set_ylabel('Normalized Flux (arbitrary units)', fontsize=15)
        max_yticks = 5
        max_xticks = 5            
        yloc = plt.MaxNLocator(max_yticks)
        xloc = plt.MaxNLocator(max_xticks)
        axarr[rownum, colnum].yaxis.set_major_locator(yloc)
        axarr[rownum, colnum].xaxis.set_major_locator(xloc)
        axarr[rownum, colnum].tick_params(axis='x', labelsize=15)
        axarr[rownum, colnum].tick_params(axis='y', labelsize=15)
                
        axarr[rownum, colnum].axhline(y=1, color='k', ls='dashed')
        
                    
        figure = plt.gcf() # get current figure
        figure.set_size_inches(20, 15)
        # when saving, specify the DPI
        #plt.savefig(plot_name, dpi = 150, bbox_inches='tight')
        pdf.savefig(figure, dpi = 150, bbox_inches='tight')
        # This is the final step: To increment!! Keep this at the end all the time, otherwise you'll get errors!!
        plt.close()
        grb_num = grb_num+1
    
    plt.show()

In [ ]:
print(line_num)

In [ ]:
plt.show()
figure = plt.gcf() # get current figure
figure.set_size_inches(20, 15)
# when saving, specify the DPI
#plt.savefig("myplot.png", dpi = 200)

In [ ]:
# Make a list of line-windows where you want to do Gaussian fits
# and enter the number of components you want to fit
# and the range of velocities over which you want to fit those components (basically the domain of the fit)

list_line_windows       = [12, 16]
list_component_num_fit  = [2, 3]
list_velocity_range_neg = [-300, -500]          #-ve end of the fit windows
list_velocity_range_pos = [500, 500]          # +ve end of the fit windows
list_line_width_guess   = [[100,100], [100,100,100]]            # 100 km/s width guess 
list_line_amp_guess     = [[1,0.5], [1,1,1]]                # amplitude guess = 1
list_line_center_guess  = [[-100, 250], [-20,50,200]]

num_windows = len(list_line_windows)


def func(x, *params):
    y = np.zeros_like(x)
    for i in range(0, len(params), 3):
        ctr = params[i]
        amp = params[i+1]
        wid = params[i+2]
        y = y + amp * np.exp( -((x - ctr)/wid)**2)
    y = 1-y
    return y

guess = []                     #guess = [0, 60000, 80, 1000, 60000, 80]
for i in range(num_windows):
    for j in range(list_component_num_fit[i]):
        guess.append(list_line_center_guess[i][j])
        guess.append(list_line_amp_guess[i][j])
        guess.append(list_line_width_guess[i][j])
    
    x = Velocity_box[list_line_windows[i]-1]
    y = Flux_box[list_line_windows[i]-1]
    error = Error_box[list_line_windows[i]-1]
    x_fit_window = x[(x>list_velocity_range_neg[i])*(x<list_velocity_range_pos[i])]
    y_fit_window = y[(x>list_velocity_range_neg[i])*(x<list_velocity_range_pos[i])]
    error_fit_window = error[(x>list_velocity_range_neg[i])*(x<list_velocity_range_pos[i])]
    
    #Check this!!! popt, pcov = curve_fit(func, x, y, p0=guess, sigma=error, absolute_sigma=True)
    popt, pcov = curve_fit(func, x_fit_window, y_fit_window, p0=guess)
    print(popt)
    
    #Extracting optimal fit parameters
    num_param = 3
    line_center_opt = []
    line_amp_opt = []
    line_width_opt = []
    
    x_fit = np.linspace(-1*delta_v, delta_v, num=500)
    fit = func(x_fit, *popt)
    
    plt.errorbar(x,y,error, ecolor='r')
    plt.title(Filename + "  " + line_name[list_line_windows[i]-1])
    plt.xlabel('Velocity (km/s)', fontsize=15)
    plt.ylabel('Normalized Flux', fontsize=15)
    plt.plot(x_fit, fit , 'g-')
    for k in range(list_component_num_fit[i]):
        line_center_opt.append(popt[num_param*k])
        center = popt[num_param*k]
        line_amp_opt.append(popt[num_param*k+1])
        line_width_opt.append(popt[num_param*k+2])
        
        plt.vlines(center,0,1, colors='k', linestyles='dashed')
        #plt.plot((center, center), (0, 1), 'k-')
    
    plt.show()
    guess[:] = []


In [ ]:
# Make a list of line-windows where you want to do Voigt fits
# and enter the number of components you want to fit
# and the range of velocities over which you want to fit those components (basically the domain of the fit)

from astropy.modeling.models import Voigt1D
from astropy.modeling import models, fitting
from astropy.modeling.models import Const1D

list_line_windows       = [51]
list_component_num_fit  = [1]
list_velocity_range_neg = [167]          #-ve end of the fit windows
list_velocity_range_pos = [195]          # +ve end of the fit windows
list_line_widthL_guess   = [[10]]            # 100 km/s width guess: Width = FWHM
list_line_widthG_guess   = [[10]]            # 100 km/s width guess: Width = FWHM 
list_line_amp_guess     = [[5]]                # amplitude guess = 1
list_line_center_guess  = [[167]]

num_windows = len(list_line_windows)

def func(x, *params):
    y = np.zeros_like(x)
    for i in range(0, len(params), 4):
        ctr = params[i]
        amp = params[i+1]
        wid_L = params[i+2]
        wid_G = params[i+3]
        v1 = Voigt1D(x_0=ctr, amplitude_L=amp, fwhm_L=wid_L, fwhm_G=wid_G)
        y = y + v1(x)
    y = 1-y
    return y

#def model_voigt(params):
#    v1 = Const1D(1.0)
#    for i in range(0, len(params), 4):
#        ctr = params[i]
#        amp = params[i+1]
#        wid_L = params[i+2]
#        wid_G = params[i+3]
#        v1 = v1 - Voigt1D(x_0=ctr, amplitude_L=amp, fwhm_L=wid_L, fwhm_G=wid_G)
#    return v1
    
guess = []                     #guess = [0, 60000, 80, 1000, 60000, 80]
for i in range(num_windows):
    for j in range(list_component_num_fit[i]):
        guess.append(list_line_center_guess[i][j])
        guess.append(list_line_amp_guess[i][j])
        guess.append(list_line_widthL_guess[i][j])
        guess.append(list_line_widthG_guess[i][j])
    
    x = Velocity_box[list_line_windows[i]-1]
    y = Flux_box[list_line_windows[i]-1]
    error = Error_box[list_line_windows[i]-1]
    x_fit_window = x[(x>list_velocity_range_neg[i])*(x<list_velocity_range_pos[i])]
    y_fit_window = y[(x>list_velocity_range_neg[i])*(x<list_velocity_range_pos[i])]
    error_fit_window = error[(x>list_velocity_range_neg[i])*(x<list_velocity_range_pos[i])]
    
    #Check this!!! popt, pcov = curve_fit(func, x, y, p0=guess, sigma=error, absolute_sigma=True)
    popt, pcov = curve_fit(func, x_fit_window, y_fit_window, p0=guess , bounds=([165,0,0,0],[169,50,200,200]))
    #print(popt)
#    fitter = fitting.LevMarLSQFitter()
#    voigt_fit = fitter(model_voigt(guess), x, y)
    

    x_fit = np.linspace(-1*delta_v, delta_v, num=500)
    dx_fit = x_fit[1]-x_fit[0]
    continuum = 1.0
    eq_width = ( continuum*(x_fit[-1] - x_fit[0]) - dx_fit*sum(func(x_fit, *popt)))/continuum
    plt.plot(x_fit, func(x_fit, *popt))   
    #plt.plot(x_fit, voigt_fit(x_fit))
    plt.errorbar(x,y,error, ecolor='r')
    plt.show()
    guess[:] = []
    print(eq_width)


In [ ]:
print(popt)

In [ ]:
print(popt)

In [ ]:
print(popt)

In [ ]:
plt.figure()
x_v = np.arange(163, 250, 0.01)
v1_v = Const1D(1.0) - Voigt1D(x_0=165, amplitude_L=2, fwhm_L=100, fwhm_G=40)
plt.plot(v1_v(x))
plt.show()

In [ ]:
    #Extracting optimal fit parameters
    num_param = 4
    line_center_opt = []
    line_amp_opt = []
    line_widthL_opt = []
    line_widthG_opt = []
    
    x_fit = np.linspace(-1*delta_v, delta_v, num=500)
    fit = func(x_fit, *popt)
    
    plt.errorbar(x,y,error, ecolor='r')
    plt.title(Filename + "  " + line_name[list_line_windows[i]-1])
    plt.xlabel('Velocity (km/s)', fontsize=15)
    plt.ylabel('Normalized Flux', fontsize=15)
    plt.plot(x_fit, fit , 'g-')
    for k in range(list_component_num_fit[i]):
        line_center_opt.append(popt[num_param*k])
        center = popt[num_param*k]
        line_amp_opt.append(popt[num_param*k+1])
        line_widthL_opt.append(popt[num_param*k+2])
        line_widthG_opt.append(popt[num_param*k+3])
        
        plt.vlines(center,0,1, colors='k', linestyles='dashed')
        #plt.plot((center, center), (0, 1), 'k-')
    
    plt.show()
    guess[:] = []
